<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/colab/Adapter_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluating Adapter 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
#drive.mount("/content/gdrive/MyDrive/master_hpi/NLP_Project/code/", force_remount=True)

Mounted at /content/gdrive/


In [3]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')



In [4]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [5]:
#!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/code/requirements.txt

In [6]:
#!pip install adapter-transformers
!pip install datasets
!pip install colorama

In [7]:
import pandas as pd
from collections import defaultdict 
import dataloader

# Dataset Loading - Stereoset

In [8]:

print(path+"dev.json")

/content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json


In [9]:
stereoset = dataloader.StereoSet(path+"dev.json")

In [10]:
stereoset

In [11]:
intersentence_examples = stereoset.get_intersentence_examples() 
intrasentence_examples = stereoset.get_intrasentence_examples()

In [12]:
intrasentence_examples[1].sentences[1].gold_label

'anti-stereotype'

In [13]:
id2term = {}
id2gold = {}
id2score = {}
example2sent = {}
domain2example = {"intersentence": defaultdict(lambda: []), "intrasentence": defaultdict(lambda: [])}

for example in intrasentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intrasentence'][example.bias_type].append(example)

for example in intersentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intersentence'][example.bias_type].append(example)

In [14]:
# When predictions are already done
#for sent in self.predictions.get('intrasentence', []) + self.predictions.get('intersentence', []):
#            self.id2score[sent['id']] = sent['score']

In [15]:
domain2example

{'intersentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntersentenceExample at 0x7f707e369310>,
              'profession': [<dataloader.IntersentenceExample at 0x7f707e36c410>,
               ...],
              'race': [<dataloader.IntersentenceExample at 0x7f7081777350>,
               ...],
              'religion': [<dataloader.IntersentenceExample at 0x7f707e180ad0>,
               <dataloader.IntersentenceExample at 0x7f707db0c810>]}),
 'intrasentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntrasentenceExample at 0x7f707ec72fd0>,
              'profession': [<dataloader.IntrasentenceExample at 0x7f70810ee550>,
               ...],
              'race': [<dataloader.IntrasentenceExample at 0x7f707ec70790>,
               ...],
              'religion': [<dataloader.IntrasentenceExample at 0x7f707ea9ae10>,
               <dataloader.IntrasentenceExample at 0x7f707e3ca290>]})}

In [16]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [17]:
!pip3 install transformers==3

     |████████████████████████████████| 757kB 10.6MB/s 
     |████████████████████████████████| 3.0MB 20.0MB/s 
ERROR: adapter-transformers 2.0.1 has requirement tokenizers<0.11,>=0.10.1, but you'll have tokenizers 0.8.0rc4 which is incompatible.
  Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Found existing installation: transformers 4.9.0.dev0
    Uninstalling transformers-4.9.0.dev0:
      Successfully uninstalled transformers-4.9.0.dev0


In [ ]:
from eval_template import *
import dataloader


class Args:
  no_cuda = False
  input_file = '/content/dev.json'
  output_file = '/content/predictions.json'
  max_seq_length = 128
  batch_size = 1

args=Args()

#class input_args:
#  def __init__(self, no_cuda, input_file, output_file, max_seq_length):
#    self.no_cuda = no_cuda
 #   self.input_file = input_file
 #   self.output_file = output_file
#    self.max_seq_length = max_seq_length

#args = input_args(False, "/content/dev.json", "/content/predictions.json", 128)
#args = {#
#    "no_cuda": False, 
#    "input_file": "/content/dev.json", 
#    "output_file":"/content/predictions.json"
#}

evaluator = BiasEvaluator(**vars(args))
results = evaluator.evaluate()

if args.output_file is not None:
  output_file = args.output_file
else:
  output_file = f"predictions_SentimentModel.json"

  output_file = os.path.join(args.output_dir, output_file)
with open(output_file, "w+") as f:
  json.dump(results, f, indent=2)


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...



---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/SentimentBert.pth
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
Maximum sequence length found: -inf



Number of parameters: 109,483,778
odict_keys(['module.bert.embeddings.word_embeddings.weight', 'module.bert.embeddings.position_embeddings.weight', 'module.bert.embeddings.token_type_embeddings.weight', 'module.bert.embeddings.LayerNorm.weight', 'module.bert.embeddings.LayerNorm.bias', 'module.bert.encoder.layer.0.attention.self.query.weight', 'module.bert.encoder.layer.0.attention.self.query.bias', 'module.bert.encoder.layer.0.attention.self.key.weight', 'module.bert.encoder.layer.0.attention.self.key.bias', 'module.bert.encoder.layer.0.attention.self.value.weight', 'module.bert.encoder.layer.0.attention.self.value.bias', 'module.bert.encoder.layer.0.attention.output.dense.weight', 'module.bert.encoder.layer.0.attention.output.dense.bias', 'module.bert.encoder.layer.0.attention.output.LayerNorm.weight', 'module.bert.encoder.layer.0.attention.output.LayerNorm.bias', 'module.bert.encoder.layer.0.intermediate.dense.weight', 'module.bert.encoder.layer.0.intermediate.dense.bias', 'module.

 60%|██████    | 3853/6369 [01:29<00:57, 43.49it/s]

In [ ]:

#!wget -P pretrained_models http://moinnadeem.com/stereoset/pretrained_models/SentimentBert.pth


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:

class Args:
  no_cuda = False
  input_file = '/content/sample_data/dev.json'
  output_file = '/content/sample_data/predictions.json'

args=Args()

In [ ]:
print(vars(args))